<a href="https://colab.research.google.com/github/YasserJxxxx/Examen_Parcial/blob/main/Protipo1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import csv
import math
import random
import time
import os
import pickle

# =======================
# 1. CARGAR DATASET
# =======================
def load_csv(filename):
    dataset = []
    with open(filename, 'r') as file:
        reader = csv.reader(file)
        header = next(reader)
        expected_cols = len(header)
        for row in reader:
            if len(row) != expected_cols:
                continue
            try:
                dataset.append(list(map(float, row)))
            except ValueError:
                continue
    print(f"Datos cargados: {len(dataset)} registros, {expected_cols} columnas.")
    return dataset

# =======================
# 2. NORMALIZAR DATA
# =======================
def normalize(dataset):
    cols = len(dataset[0]) - 1
    mins = [min(row[i] for row in dataset) for i in range(cols)]
    maxs = [max(row[i] for row in dataset) for i in range(cols)]
    for row in dataset:
        for i in range(cols):
            if maxs[i] - mins[i] != 0:
                row[i] = (row[i] - mins[i]) / (maxs[i] - mins[i])
    return dataset

# =======================
# 3. DIVIDIR DATA
# =======================
def split_data(dataset, train_ratio=0.8):
    random.shuffle(dataset)
    split_point = int(len(dataset) * train_ratio)
    return dataset[:split_point], dataset[split_point:]

# =======================
# 4. MODELO PERCEPTRÓN
# =======================
class Perceptron:
    def __init__(self, n_inputs, learning_rate=0.05):
        self.weights = [random.uniform(-1, 1) for _ in range(n_inputs + 1)]
        self.learning_rate = learning_rate
        self.history = []

    def activate(self, inputs):
        total = self.weights[-1]
        for i in range(len(inputs)):
            total += self.weights[i] * inputs[i]
        return 1.0 if total >= 0 else 0.0

    def train(self, training_data, n_epochs=10):
        start_time = time.time()
        for epoch in range(1, n_epochs + 1):
            total_error = 0
            correct = 0
            for row in training_data:
                inputs = row[:-1]
                expected = row[-1]
                prediction = self.activate(inputs)
                error = expected - prediction
                total_error += abs(error)
                if prediction == expected:
                    correct += 1
                # actualización de pesos
                for i in range(len(inputs)):
                    self.weights[i] += self.learning_rate * error * inputs[i]
                self.weights[-1] += self.learning_rate * error
            accuracy = correct / len(training_data) * 100
            self.history.append({'epoch': epoch, 'error': total_error, 'accuracy': accuracy})
            print(f"Época {epoch}/{n_epochs} | Error: {total_error} | Precisión: {accuracy:.2f}%")
        print(f"Tiempo total de entrenamiento: {time.time() - start_time:.2f}s")

    def predict(self, inputs):
        return self.activate(inputs)

# =======================
# 5. MÉTRICAS
# =======================
def evaluate_metrics(model, test_data):
    TP = TN = FP = FN = 0
    start_eval = time.time()
    for row in test_data:
        inputs = row[:-1]
        expected = row[-1]
        pred = model.predict(inputs)
        if expected == 1 and pred == 1:
            TP += 1
        elif expected == 0 and pred == 0:
            TN += 1
        elif expected == 0 and pred == 1:
            FP += 1
        elif expected == 1 and pred == 0:
            FN += 1
    end_eval = time.time()
    total = TP + TN + FP + FN
    accuracy = (TP + TN) / total if total else 0
    precision = TP / (TP + FP) if (TP + FP) else 0
    recall = TP / (TP + FN) if (TP + FN) else 0
    specificity = TN / (TN + FP) if (TN + FP) else 0
    f1_score = 2 * precision * recall / (precision + recall) if (precision + recall) else 0
    balanced_accuracy = (recall + specificity) / 2
    fpr = FP / (FP + TN) if (FP + TN) else 0
    tnr = specificity

    print("\n=== RESULTADOS DEL MODELO ===")
    print(f"Accuracy: {accuracy*100:.2f}%")
    print(f"Precision (Fraude): {precision*100:.2f}%")
    print(f"Recall (Sensibilidad): {recall*100:.2f}%")
    print(f"F1 Score: {f1_score*100:.2f}%")
    print(f"Specificity (Legítimas): {specificity*100:.2f}%")
    print(f"Balanced Accuracy: {balanced_accuracy*100:.2f}%")
    print(f"False Positive Rate: {fpr*100:.2f}%")
    print(f"True Negative Rate: {tnr*100:.2f}%")
    print(f"Tiempo promedio por registro: {(end_eval - start_eval)/total:.6f}s")

    print("\n--- MATRIZ DE CONFUSIÓN ---")
    print("               Predicho: NoFraude   Predicho: Fraude")
    print(f"Real: NoFraude       {TN:5d}             {FP:5d}")
    print(f"Real: Fraude         {FN:5d}             {TP:5d}")

# =======================
# 6. GRAFICAR ENTRENAMIENTO
# =======================
def plot_history(history):
    print("\n--- EVOLUCIÓN DEL ENTRENAMIENTO ---")
    print("Época | Error | Precisión | Visualización")
    for h in history:
        bar_acc = "#" * int(h['accuracy'] / 2)
        bar_err = "-" * int(50 - (h['accuracy'] / 2))
        print(f"{h['epoch']:>3} | {h['error']:>5} | {h['accuracy']:6.2f}% | {bar_acc}{bar_err}")

# =======================
# 7. GUARDAR Y CARGAR MODELO
# =======================
def save_model(model, filename="perceptron_model.pkl"):
    with open(filename, "wb") as f:
        pickle.dump(model, f)

def load_model(filename="perceptron_model.pkl"):
    if os.path.exists(filename):
        with open(filename, "rb") as f:
            return pickle.load(f)
    return None

# =======================
# 8. MENÚ PRINCIPAL
# =======================
def main():
    while True:
        print("\n=== MENÚ PRINCIPAL ===")
        print("1. Evaluar modelo con dataset")
        print("2. Insertar nuevos datos para predicción")
        print("3. Salir del programa")
        opcion = input("Seleccione una opción (1-3): ").strip()

        if opcion == "1":
            print("\nCargando dataset...")
            dataset = load_csv("creditcard.csv")
            print("Normalizando datos (0..1)...")
            dataset = normalize(dataset)
            train_data, test_data = split_data(dataset)

            perceptron = load_model()
            if perceptron:
                print("\nModelo cargado. Reentrenando brevemente para ajustar...")
                perceptron.train(train_data, n_epochs=2)
            else:
                perceptron = Perceptron(len(train_data[0]) - 1)
                print("\nEntrenando nuevo modelo...")
                perceptron.train(train_data, n_epochs=10)

            evaluate_metrics(perceptron, test_data)
            plot_history(perceptron.history)
            save_model(perceptron)
            print("\n✅ Modelo guardado exitosamente.")

        elif opcion == "2":
            perceptron = load_model()
            if not perceptron:
                print("❌ No hay modelo entrenado. Ejecute primero la opción 1.")
                continue
            print("\nIngrese valores normalizados entre 0 y 1 (x para volver al menú):")
            n_inputs = len(perceptron.weights) - 1
            while True:
                entrada = input(f"Ingrese {n_inputs} valores separados por espacio: ").strip()
                if entrada.lower() == "x":
                    break
                valores = entrada.split()
                if len(valores) != n_inputs:
                    print(f"❌ Se esperaban {n_inputs} valores. Intente de nuevo.\n")
                    continue
                try:
                    valores = list(map(float, valores))
                except ValueError:
                    print("❌ Error: todos los valores deben ser numéricos.\n")
                    continue
                resultado = perceptron.predict(valores)
                if resultado == 1:
                    print("⚠️ Posible FRAUDE detectado.\n")
                else:
                    print("✅ Transacción legítima detectada.\n")

        elif opcion == "3":
            print("\n👋 Saliendo del programa. ¡Hasta pronto!")
            break
        else:
            print("❌ Opción inválida. Intente nuevamente.")

# =======================
# 9. EJECUCIÓN
# =======================
if __name__ == "__main__":
    main()



=== MENÚ PRINCIPAL ===
1. Evaluar modelo con dataset
2. Insertar nuevos datos para predicción
3. Salir del programa
Seleccione una opción (1-3): 1

Cargando dataset...
Datos cargados: 122974 registros, 31 columnas.
Normalizando datos (0..1)...

Modelo cargado. Reentrenando brevemente para ajustar...
Época 1/2 | Error: 265.0 | Precisión: 99.73%
Época 2/2 | Error: 230.0 | Precisión: 99.77%
Tiempo total de entrenamiento: 1.70s

=== RESULTADOS DEL MODELO ===
Accuracy: 99.80%
Precision (Fraude): 78.43%
Recall (Sensibilidad): 51.95%
F1 Score: 62.50%
Specificity (Legítimas): 99.96%
Balanced Accuracy: 75.95%
False Positive Rate: 0.04%
True Negative Rate: 99.96%
Tiempo promedio por registro: 0.000007s

--- MATRIZ DE CONFUSIÓN ---
               Predicho: NoFraude   Predicho: Fraude
Real: NoFraude       24507                11
Real: Fraude            37                40

--- EVOLUCIÓN DEL ENTRENAMIENTO ---
Época | Error | Precisión | Visualización
  1 |   8.0 |  99.83% | ######################